# 문제정의
- 영화 리뷰 데이터셋을 활용해서 긍정리뷰와 부정리뷰를 구분
- 긍정 / 부정 리뷰에서 자주 사용되는 단어 확인

# 데이터 수집
- Large movie dataset 다운로드

In [1]:
# 파일을 읽어오는 함수
from sklearn.datasets import load_files

In [4]:
train_data_url = 'data/aclimdb/train'
test_data_url = 'data/aclimdb/test'

In [5]:
reviews_train = load_files(train_data_url , shuffle = True)
reviews_test = load_files(test_data_url , shuffle = True)

In [7]:
reviews_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [9]:
len(reviews_train.data)

25000

In [10]:
reviews_train.data[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

In [11]:
# 0이 부정 1이 긍정
reviews_train.target

array([1, 0, 1, ..., 0, 0, 0])

# 데이터 전처리

In [12]:
reviews_train.data[0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

## 태그 제거
- br 태그를 띄어쓰기로 변경

In [13]:
reviews_train.data[0].replace(b'</br>' , b' ')

TypeError: a bytes-like object is required, not 'str'